# ProfileScope: Social Media Profile Analysis Demo

This notebook demonstrates how to use ProfileScope to analyze social media profiles and interpret the results.

## Setup and Installation

First, let's import the required modules and set up our environment.

In [ ]:
import sys
import os
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the project root to the path so we can import our modules
sys.path.append(os.path.abspath('..'))

# Import ProfileScope modules
from app.core.analyzer import SocialMediaAnalyzer

# Set plot style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

## Basic Usage: Simple Test

Let's create a simple example to verify that our analyzer works properly:

In [ ]:
# Initialize the analyzer
analyzer = SocialMediaAnalyzer()

# Print config to verify
print("Analyzer configuration:")
print(json.dumps(analyzer.config, indent=2))

In [ ]:
# Define a test profile
platform = "twitter"
profile_id = "test_user"

# Perform a test analysis
results = analyzer.analyze_profile(platform, profile_id)

# Print results summary
print(f"Analysis completed for {profile_id} on {platform}")
print(f"Analysis date: {results['metadata']['analysis_date']}")
print(f"Analyzer version: {results['metadata']['analyzer_version']}")

## Exploring the Results

Now let's explore what's in the analysis results:

In [ ]:
# Let's see what top-level keys are in the results
print("Results structure:")
for key in results.keys():
    print(f"- {key}")

## Creating Visualizations

Let's create some simple visualizations of our results:

In [ ]:
# Create a simple bar chart of personality traits
if 'personality_traits' in results['content_analysis']:
    traits = results['content_analysis']['personality_traits']
    
    # Convert to DataFrame for easier plotting
    traits_df = pd.DataFrame(list(traits.items()), columns=['Trait', 'Score'])
    traits_df = traits_df.sort_values('Score', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Score', y='Trait', data=traits_df)
    plt.title('Personality Traits Analysis')
    plt.xlim(0, 1)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize authenticity components if available
if 'authenticity_analysis' in results and 'overall_authenticity' in results['authenticity_analysis']:
    auth = results['authenticity_analysis']['overall_authenticity']
    
    # Create a pie chart
    labels = ['Authentic', 'Risk']
    sizes = [auth['score'], 1 - auth['score']]
    colors = ['#66b3ff', '#ff9999']
    explode = (0.1, 0)  # explode the first slice for emphasis
    
    plt.figure(figsize=(8, 8))
    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=90)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.title('Profile Authenticity Analysis')
    plt.show()

## Exporting Results

Let's export our analysis results for later use or sharing:

In [ ]:
# Create output directory
output_dir = "../output"
os.makedirs(output_dir, exist_ok=True)

# Generate output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"{output_dir}/{platform}_{profile_id}_{timestamp}.json"

# Export the results
analyzer.export_results(results, output_file)
print(f"Results exported to: {output_file}")

## Project Maintenance

ProfileScope includes utilities for project maintenance, such as the cleanup script. Here's how to use it:

In [ ]:
# Example of running the cleanup script from Python
import subprocess

def run_cleanup(verbose=True):
    """Run the ProfileScope cleanup script"""
    script_path = os.path.join(os.path.abspath('..'), 'scripts', 'cleanup.sh')
    
    if not os.path.exists(script_path):
        print(f"Cleanup script not found at: {script_path}")
        return False
    
    try:
        result = subprocess.run(['bash', script_path], 
                               capture_output=True, 
                               text=True, 
                               check=True)
        if verbose:
            print("Cleanup completed successfully")
            print("Output:")
            print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error running cleanup script: {e}")
        if verbose:
            print("Error output:")
            print(e.stderr)
        return False

# Uncomment to run cleanup (use carefully!)
# run_cleanup()

### Running Tests with Automatic Cleanup

ProfileScope now supports automatically running the cleanup script before tests. Let's see how to use it:

In [ ]:
# Run tests with automatic cleanup
def run_tests_with_cleanup():
    """Run tests with automatic cleanup first"""
    script_path = os.path.join(os.path.abspath('..'), 'run_tests.py')
    
    if not os.path.exists(script_path):
        print(f"Test runner script not found at: {script_path}")
        return False
    
    try:
        print("Running tests with automatic cleanup...")
        result = subprocess.run([sys.executable, script_path], 
                              capture_output=True, 
                              text=True)
        
        print("Test output:")
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ All tests passed!")
            return True
        else:
            print("\n❌ Some tests failed.")
            if result.stderr:
                print("Error output:")
                print(result.stderr)
            return False
    except Exception as e:
        print(f"Error running tests: {e}")
        return False

# Uncomment to run tests with automatic cleanup
# run_tests_with_cleanup()

### Preventing Duplicate Cleanup Runs

Our testing system now prevents cleanup from running twice by using an environment variable to track whether cleanup has already run:

In [ ]:
# Example of checking and managing cleanup status
def check_cleanup_status():
    """Check and manage cleanup status"""
    if os.environ.get("PROFILESCOPE_CLEANUP_RUN") == "1":
        print("Cleanup has already been run in this session")
        return True
    else:
        print("Cleanup has not been run yet")
        return False

def reset_cleanup_status():
    """Reset cleanup status to force running it again"""
    if "PROFILESCOPE_CLEANUP_RUN" in os.environ:
        del os.environ["PROFILESCOPE_CLEANUP_RUN"]
        print("Cleanup status reset - will run again on next test")
    else:
        print("Cleanup status was not set")

# Check current cleanup status
check_cleanup_status()

### Using Alternative Test Runners

In case you encounter issues with pytest plugins like pytest-html, we provide alternative test runners:

In [ ]:
# Run tests with simple XML report (more reliable than HTML reports)
def run_simple_tests():
    """Run tests with simple XML reporting"""
    script_path = os.path.join(os.path.abspath('..'), 'run_simple_tests.py')
    
    if not os.path.exists(script_path):
        print(f"Simple test runner script not found at: {script_path}")
        return False
    
    try:
        print("Running simplified tests...")
        result = subprocess.run([sys.executable, script_path], 
                              capture_output=True, 
                              text=True)
        
        print("Test output:")
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ All tests passed!")
            return True
        else:
            print("\n❌ Some tests failed.")
            if result.stderr:
                print("Error output:")
                print(result.stderr)
            return False
    except Exception as e:
        print(f"Error running tests: {e}")
        return False

# Run tests directly with pytest
def run_direct_pytest():
    """Run tests directly with pytest"""
    try:
        print("Running pytest directly...")
        # First run cleanup
        if not check_cleanup_status():
            run_cleanup(verbose=False)
        
        # Run pytest directly
        result = subprocess.run([sys.executable, '-m', 'pytest'], 
                              capture_output=True, 
                              text=True,
                              cwd=os.path.abspath('..'))
        
        print("Test output:")
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ All tests passed!")
            return True
        else:
            print("\n❌ Some tests failed.")
            if result.stderr:
                print("Error output:")
                print(result.stderr)
            return False
    except Exception as e:
        print(f"Error running tests: {e}")
        return False

# Use run_direct_pytest() or run_simple_tests() if you encounter issues with the standard test runner

## Conclusion

In this notebook, we've demonstrated how to:

1. Set up and initialize the ProfileScope analyzer
2. Perform a basic profile analysis
3. Explore and visualize the results
4. Export the results for sharing or further analysis
5. Perform maintenance tasks like cleanup
6. Run tests with automatic cleanup
7. Manage cleanup run status to prevent duplicate runs
8. Use alternative test runners for better compatibility

To learn more about ProfileScope's features and capabilities, check out the documentation and additional examples.